In [ ]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import grangercausalitytests, adfuller

df_sent = pd.read_csv("daily_sentiment_aggregated.csv")
# Wir filtern für EIN Unternehmen mit vielen News (Beispiel: Siemens oder SAP)
# Schauen Sie in Ihr CSV, wer die meisten Treffer hat.
target_company_news_name = "Siemens" # Passen Sie den String exakt an Ihre CSV an!
target_ticker = "SIE.DE" # Das passende Kürzel für yfinance

print(f"Analysiere: {target_company_news_name} vs. {target_ticker}")

# Sentiment Daten filtern & Index setzen
df_company_sent = daily_sentiment[daily_sentiment['Firma'].str.contains(target_company_news_name, case=False, na=False)].copy()
df_company_sent['Datum'] = pd.to_datetime(df_company_sent['Datum'])
df_company_sent.set_index('Datum', inplace=True)

# 2. Aktienkurse laden (yfinance)
# Zeitraum: Wir nehmen einen großen Zeitraum, um sicher zu sein
stock_data = yf.download(target_ticker, start="2023-01-01", end="2025-12-31")

# WICHTIG: Wir brauchen die Rendite (Returns), nicht den absoluten Preis!
# Zeitreihenmodelle brauchen "stationäre" Daten. Preise haben Trends, Renditen meist nicht.
stock_data['Returns'] = stock_data['Close'].pct_change()

# 3. Mergen (Join)
# Wir verbinden Aktien-Daten (Links) mit Sentiment (Rechts) über das Datum
# 'inner' join: Wir behalten nur Tage, an denen BEIDES existiert (Handelstag + News)
# Alternativ 'left' join auf Aktien, und fehlende News mit 0 füllen.

df_model = stock_data[['Returns']].join(df_company_sent['Sentiment_Score'], how='left')

# Fehlende Werte behandeln
# Wenn es keine News gab, ist der Sentiment-Impact 0 (neutral)
df_model['Sentiment_Score'] = df_model['Sentiment_Score'].fillna(0)
# Wochenenden (wo Börse zu ist) droppen wir, da wir dort keine Rendite haben
df_model.dropna(inplace=True)

print("Daten für Modellierung bereit:")
print(df_model.head())

In [ ]:
def adf_check(time_series, name):
    result = adfuller(time_series)
    print(f"Augmented Dickey-Fuller Test für {name}:")
    print(f"P-Wert: {result[1]}")
    if result[1] <= 0.05:
        print("✅ Daten sind stationär. Perfekt für Analyse.\n")
    else:
        print("⚠️ Daten sind NICHT stationär. Differenzierung nötig.\n")

adf_check(df_model['Returns'], "Aktienrendite")
adf_check(df_model['Sentiment_Score'], "News Sentiment")

In [ ]:
# Wir testen bis zu 5 Tage Verzögerung (Lags)
max_lags = 5

# Format für statsmodels: Spalte 1 = Ziel (Y), Spalte 2 = Prädiktor (X)
data_for_test = df_model[['Returns', 'Sentiment_Score']]

print(f"Führe Granger-Kausalitätstest durch (Max Lags: {max_lags})...")
print("Nullhypothese: Sentiment verursacht KEINE Kursänderung.\n")

# Der Test
gc_res = grangercausalitytests(data_for_test, max_lags, verbose=True)

# Interpretationshilfe für das Output-Fenster:
# Suchen Sie nach Zeilen wie: "ssr_ftest" und dem "p=" Wert.
# p < 0.05 bedeutet: Statistisch signifikant! (Wir verwerfen die Nullhypothese)
# Das heißt: Sentiment HAT Einfluss.

In [ ]:
from statsmodels.tsa.api import VAR

# 1. Modell erstellen
model = VAR(df_model)

# 2. Modell fitten (automatische Lag-Wahl basierend auf AIC)
results = model.fit(maxlags=5, ic='aic')
print(results.summary())

# 3. Impulse Response Function (IRF) plotten
# Das zeigt: Was passiert mit der Aktie ("Returns"), wenn das Sentiment ("Sentiment_Score") einen Schock bekommt?
irf = results.irf(10) # Schau 10 Tage in die Zukunft

plt.figure(figsize=(10,6))
# Wir wollen sehen: Response of Returns to Sentiment_Score
irf.plot(orth=False, response='Returns', impulse='Sentiment_Score')
plt.title("Wie reagiert die Aktienrendite auf einen Sentiment-Schock?")
plt.show()